## Appendix A

In [1]:
#user input data
age = int(input("Please enter your age(integer):"))
gender = input("Please enter your gender(male/female):")
gender = gender+'_'
act = input("Please choose your physical activity level(sedentary/moderately_active/active):")

Please enter your age(integer):22
Please enter your gender(male/female):female
Please choose your physical activity level(sedentary/moderately_active/active):sedentary


In [2]:
#read in user data
if 18 <= age <=25:
    age = '18-25_'
elif 26 <= age <= 30:
    age = '26-30_'
elif age > 30:
    age = 'over_30_'
user = age + gender + act
user

'18-25_female_sedentary'

In [3]:
from gurobipy import *
import numpy as np
import pandas as pd
constraints = pd.read_excel('OR Project Data Constraints.xlsx', index_col = 0, header= 0)
n,m = constraints.shape

In [4]:
req= constraints[user]

In [5]:
#########Parameters Set-up############
# Read the data from the csv file and use the first column as the index of rows

project = pd.read_csv('data.csv')
N,M = project.shape
print(project.columns)

Index(['Category', 'Food Name', 'Energy (kcal)', 'Protein (g)',
       'Total fat (g)', 'Saturated fat (g)', 'Dietary fibre (g)',
       'Carbohydrate (g)', 'Cholesterol (mg)', 'Sodium (mg)', 'Sugar(g)'],
      dtype='object')


In [6]:
project.shape

(136, 11)

In [7]:
nutrient = project.iloc[0:N, 2:M] 
nutrient.head()

,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
0,323.71,28.11,23.43,9.37,0.00,0.00,153.34,1316.15,0.00
1,318.04,25.14,7.00,1.85,1.74,36.57,76.03,1969.63,4.63
2,556.47,13.16,26.74,10.05,1.54,65.77,205.80,1937.08,13.62
3,695.50,27.30,39.65,14.24,3.90,57.85,104.00,7904.00,2.18
4,668.10,18.60,51.09,20.70,0.00,33.54,366.80,772.90,2.16


In [8]:
#for objective function
total_fat = nutrient.iloc[0:N,2] 
saturated_fat = nutrient.iloc[0:N,3] 
cholesterol = nutrient.iloc[0:N,6] 
sodium = nutrient.iloc[0:N,7] 
sugar = nutrient.iloc[0:N,8] 

In [9]:
req

Nutrient
Energy (Kcal)        1600.000000
Protein (g)            44.000000
Total fat              64.000000
Saturated fat          20.000000
Fiber (g)              23.000000
Carbohydrate (g)      120.000000
Cholesterol           250.000000
Sodium               2200.000000
sugar                  41.375743
Name: 18-25_female_sedentary, dtype: float64

In [10]:
m = Model("food")
D=5

# Creat variables
x = m.addVars(N, D, vtype=GRB.BINARY, name = "x")

# Set objective
#for penalty function
#m.setObjective(quicksum(x[i,d]*total_fat[i]/req[2]+x[i,d]*saturated_fat[i]/req[3]+x[i,d]*cholesterol[i]/req[6]+x[i,d]*sodium[i]/req[7]+x[i,d]*sugar[i]/req[8] for i in range(N) for d in range(D)), GRB.MINIMIZE)
#minimize total fat
#m.setObjective(quicksum(x[i,d]*total_fat[i] for i in range(N) for d in range(D)), GRB.MINIMIZE)
#minimize saturated_fat
#m.setObjective(quicksum(x[i,d]*saturated_fat[i] for i in range(N) for d in range(D)), GRB.MINIMIZE)
#minimize cholesterol
#m.setObjective(quicksum(x[i,d]*cholesterol[i] for i in range(N) for d in range(D)), GRB.MINIMIZE)
#minimize sodium
#m.setObjective(quicksum(x[i,d]*sodium[i] for i in range(N) for d in range(D)), GRB.MINIMIZE)
#minimize sugar
m.setObjective(quicksum(x[i,d]*sugar[i] for i in range(N) for d in range(D)), GRB.MINIMIZE)
    
# Add constraints:
for d in range(D):
    m.addConstr( quicksum(x[i,d] for i in project[project.Category == 'Main'].index) ==2, name = "mains") 
    m.addConstr( quicksum(x[i,d] for i in project[project.Category == 'Breakfast'].index) ==1, name = "breakfast")
    m.addConstr( quicksum(x[i,d] for i in  project[project.Category == 'Fruit'].index) <=2, name = "fruits")
    m.addConstr( quicksum(x[i,d] for i in  project[project.Category == 'Dessert'].index) <=1, name = "dessert")
    m.addConstr( quicksum(x[i,d] for i in  project[project.Category == 'Beverage'].index) <=2, name = "bev")

    m.addConstrs( (quicksum(x[i,d]*nutrient.iloc[i, j] for i in range(N)) >= req[j] for j in [0,1,4,5])
             , name = "min_req")
    m.addConstrs( (quicksum(x[i,d]*nutrient.iloc[i, j] for i in range(N)) <= req[j] for j in [2,3,6,7])
             , name = "max_intake")

for i in range(N):
    m.addConstr( quicksum(x[i,d] for d in range(D)) <=2, name = "max_repitition") 

Academic license - for non-commercial use only


In [11]:
project.iloc[1,2]

318.04

In [ ]:
# Solving the model
m.setParam( 'OutputFlag', False )
%time m.optimize()

# #  Print optimal solutions and optimal value
# for v in m.getVars():
#     if(v.x == 1):
#         print(v.VarName, v.x)   

# print('Obj:', m.objVal)

In [ ]:
#  Print optimal x for x nonzero and optimal value
s_edge = []
for v in x:    
    if x[v].x > 0.001:
        print(x[v].VarName, x[v].x)
        #add both of the indicies by 1
        edge = np.add(v, (1,1))
        #append the edge to the resulting list of edges
        s_edge.append(edge)

print('Obj:', m.objVal)
print(s_edge)

In [12]:
project.iloc[[37,48,59,86,100,104,116]]

,Category,Food Name,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
37,Main,"Chicken chop, grilled",679.00,51.50,30.58,12.08,6.97,49.87,76.4,655.20,10.23
48,Main,Beef and Teriyaki Chicken Bento,743.28,43.03,20.88,7.92,10.76,95.35,0.0,1515.90,4.40
59,Dessert & Snack,Vietnam spring roll,78.95,4.53,1.80,0.69,0.43,11.14,0.0,104.04,2.17
86,Beverage,"Soya bean milk, without sugar",82.50,6.75,5.00,0.65,2.00,3.00,0.0,30.00,2.75
100,Fruit,Dragonfruit,105.84,2.81,0.22,0.04,5.83,22.90,0.0,25.92,14.26
104,Fruit,Mandarin orange,38.54,0.96,0.26,0.04,1.58,7.97,0.0,2.63,0.79
116,Breakfast,"Red rice porridge, plain",74.24,2.30,0.51,0.15,1.02,15.10,0.0,7.68,0.00


In [16]:
project.iloc[[ 14, 36, 58, 59, 63, 67, 102, 104, 116]]

,Category,Food Name,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
14,Main,Nasi Lemak with chicken wing,656.58,26.08,24.94,10.65,6.52,81.21,117.44,877.09,0.80
36,Main,Salmon and Teriyaki Chicken Bento,664.54,28.54,16.99,5.17,8.03,99.90,0.00,1097.16,5.35
58,Dessert & Snack,Pan-fried turnip cake,61.64,0.71,3.73,1.41,0.41,6.35,4.14,139.61,1.50
59,Dessert & Snack,Vietnam spring roll,78.95,4.53,1.80,0.69,0.43,11.14,0.00,104.04,2.17
63,Dessert & Snack,Beancurd Dessert with Red Bean Toppings,164.64,9.80,5.25,2.63,7.45,19.60,0.00,50.18,10.58
67,Dessert & Snack,"Biscuit, macaroon",46.36,0.48,2.43,1.75,0.82,5.44,0.00,22.23,3.37
102,Fruit,Passion fruit,17.65,0.40,0.13,0.01,1.89,4.26,0.00,5.10,2.04
104,Fruit,Mandarin orange,38.54,0.96,0.26,0.04,1.58,7.97,0.00,2.63,0.79
116,Breakfast,"Red rice porridge, plain",74.24,2.30,0.51,0.15,1.02,15.10,0.00,7.68,0.00


In [13]:
project.iloc[[47, 48, 76, 82, 91, 102, 105, 115]]

,Category,Food Name,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
47,Main,"French fries, medium, McDonald's",373.52,3.48,19.00,7.60,4.00,47.00,0.00,290.00,0.23
48,Main,Beef and Teriyaki Chicken Bento,743.28,43.03,20.88,7.92,10.76,95.35,0.00,1515.90,4.40
76,Dessert & Snack,Yoghurt muesli bar,123.38,2.05,4.31,2.88,1.98,18.45,3.72,22.94,7.16
82,Dessert & Snack,"Snack bar, fruit, nuts, seeds",204.00,4.55,9.80,3.05,3.85,25.20,0.00,45.50,18.05
91,Beverage,"Milk, low fat",97.50,9.00,1.50,1.08,0.75,12.00,7.50,100.00,8.00
102,Fruit,Passion fruit,17.65,0.40,0.13,0.01,1.89,4.26,0.00,5.10,2.04
105,Fruit,Orange,46.48,1.11,0.11,0.00,2.66,8.88,0.00,3.33,8.88
115,Breakfast,Plain porridge,207.48,6.55,1.64,0.00,2.18,42.04,0.00,382.20,0.00


In [14]:
project.iloc[[21, 36, 63, 66, 67, 76, 86, 135]]


,Category,Food Name,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
21,Main,Chinese rojak,517.58,16.29,21.48,6.06,6.84,63.85,12.36,914.32,0.00
36,Main,Salmon and Teriyaki Chicken Bento,664.54,28.54,16.99,5.17,8.03,99.90,0.00,1097.16,5.35
63,Dessert & Snack,Beancurd Dessert with Red Bean Toppings,164.64,9.80,5.25,2.63,7.45,19.60,0.00,50.18,10.58
66,Dessert & Snack,"Biscuit, jam-filled",59.78,0.62,2.48,1.22,0.25,8.86,0.00,19.60,4.54
67,Dessert & Snack,"Biscuit, macaroon",46.36,0.48,2.43,1.75,0.82,5.44,0.00,22.23,3.37
76,Dessert & Snack,Yoghurt muesli bar,123.38,2.05,4.31,2.88,1.98,18.45,3.72,22.94,7.16
86,Beverage,"Soya bean milk, without sugar",82.50,6.75,5.00,0.65,2.00,3.00,0.00,30.00,2.75
135,Breakfast,"Pow, char siew, steamed",160.59,5.78,6.47,2.44,0.74,19.77,10.07,208.82,0.00


In [15]:
project.iloc[[14, 21, 58, 82, 88, 100, 105, 115]]

,Category,Food Name,Energy (kcal),Protein (g),Total fat (g),Saturated fat (g),Dietary fibre (g),Carbohydrate (g),Cholesterol (mg),Sodium (mg),Sugar(g)
14,Main,Nasi Lemak with chicken wing,656.58,26.08,24.94,10.65,6.52,81.21,117.44,877.09,0.80
21,Main,Chinese rojak,517.58,16.29,21.48,6.06,6.84,63.85,12.36,914.32,0.00
58,Dessert & Snack,Pan-fried turnip cake,61.64,0.71,3.73,1.41,0.41,6.35,4.14,139.61,1.50
82,Dessert & Snack,"Snack bar, fruit, nuts, seeds",204.00,4.55,9.80,3.05,3.85,25.20,0.00,45.50,18.05
88,Beverage,Brewed coffee,3.70,0.16,0.00,0.00,0.00,0.62,0.00,3.10,0.60
100,Fruit,Dragonfruit,105.84,2.81,0.22,0.04,5.83,22.90,0.00,25.92,14.26
105,Fruit,Orange,46.48,1.11,0.11,0.00,2.66,8.88,0.00,3.33,8.88
115,Breakfast,Plain porridge,207.48,6.55,1.64,0.00,2.18,42.04,0.00,382.20,0.00
